# Analyzing where the big trace lengths are coming from

In [55]:
import os
import ntpath
import glob
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from os import listdir
from os.path import isfile, join

from distributed_ledger_process_miner.parser import event_log_parser as parser
from distributed_ledger_process_miner.miner import heuristic_miner as miner

pd.options.display.max_rows = 4000
pd.options.display.max_seq_items = 2000

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [2]:
path_to_raw_transaction_bulk = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_data'
path_contracts_lookup = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/contractsWithERCFlags.csv'
path_block_times = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/blockTimes.csv'
output_path_prefix = 'out'
extension = 'csv'

In [3]:
contracts_lookup = pd.read_csv(path_contracts_lookup)
block_times = pd.read_csv(path_block_times)

In [5]:
transaction_file_names = [
    'ps_transactions5400000-5409999_event_log.csv',
    'ps_transactions5410000-5419999_event_log.csv',
    'ps_transactions5420000-5429999_event_log.csv',
    'ps_transactions5430000-5439999_event_log.csv',
    'ps_transactions5440000-5449999_event_log.csv',
    'ps_transactions5450000-5459999_event_log.csv',
    'ps_transactions5460000-5469999_event_log.csv',
    'ps_transactions5470000-5479999_event_log.csv',
    'ps_transactions5480000-5489999_event_log.csv',
    'ps_transactions5490000-5499999_event_log.csv',
    'ps_transactions5500000-5500000_event_log.csv'    
]

In [6]:
transaction_prefix_path = '/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/'

We examine one of our transaction event logs as an example and see what is the biggest trace inside it

In [9]:
df_1 = pd.read_csv(transaction_prefix_path + transaction_file_names[1])

In [10]:
df_1.head()

,Unnamed: 0,total_pos,action.input,senderIsContract,senderIsERC20,sender_id,receiverIsContract,receiverIsERC20,receiver_id,timestamp,transaction_id,sender_type,receiver_type,transaction_type
0,0,5410000000000000,0x,False,False,3006527,False,False,922411,1523288469,8085075,U,U,UtU
1,1,5410000000000001,0x,False,False,1689407,False,False,2501199,1523288469,6377519,U,U,UtU
2,2,5410000000000002,0xa9059cbb000000000000000000000000ff8c7361b932...,False,False,346437,True,True,2724134,1523288469,8138524,C,C,UtC
3,3,5410000000000003,0x,False,False,2957845,False,False,1113693,1523288469,2610496,U,U,UtU
4,4,5410000000000004,0x,False,False,1927859,False,False,616723,1523288469,38583,U,U,UtU


In [13]:
df_1.groupby('transaction_id').count().sort_values(by='total_pos', ascending=False).head(20)

,Unnamed: 0,total_pos,action.input,senderIsContract,senderIsERC20,sender_id,receiverIsContract,receiverIsERC20,receiver_id,timestamp,sender_type,receiver_type,transaction_type
transaction_id,,,,,,,,,,,,,
3560127,601,601,601,601,601,601,601,601,601,601,601,601,601
1844628,601,601,601,601,601,601,601,601,601,601,601,601,601
9786439,481,481,481,481,481,481,481,481,481,481,481,481,481
1226838,481,481,481,481,481,481,481,481,481,481,481,481,481
2031272,481,481,481,481,481,481,481,481,481,481,481,481,481
2081526,451,451,451,451,451,451,451,451,451,451,451,451,451
452483,451,451,451,451,451,451,451,451,451,451,451,451,451
5351658,451,451,451,451,451,451,451,451,451,451,451,451,451
11647044,451,451,451,451,451,451,451,451,451,451,451,451,451


Apparently the biggest trace is 601. We can use the transaction id and look up the hex value in our transaction lookup table

In [ ]:
transaction_lookup = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/transaction_lookup.csv')

In [15]:
transaction_lookup.head()

,transaction_hash,id
0,NaN,0
1,0x51c2e3c823af900a1b419a3087c770f9189495afa78b...,1
2,0xf727717af695fea46fb29be679929eac7bbb2c9a5951...,2
3,0xc8fd61f38369918f305a86983215e1100f8a19f71fc1...,3
4,0xe8c6ecd65e7916c3d2109b2524ae4632a8dc71778f8d...,4


In [56]:
transaction_lookup[transaction_lookup['id'] == 3560127]

,transaction_hash,id
3560127,0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,3560127


Now that we found the transaction hash where the 601 subtraces belong to, we can look the transaction up in our raw transaction table and examine what is going on there.

In [57]:
raw_input = pd.read_csv('/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/parity_transactions/transactions5410000-5419999.csv')

/Users/marcelmuller/Documents/Uni/Master/Semester_9_SS_18/Masterarbeit/distributed_ledger_process_miner/dl-pm/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,1,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [58]:
a = raw_input[raw_input['transactionHash']=='0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba']

In [60]:
for column in a.columns:
    print("column "+column+" unique values: \n"+str(a[column].unique()))

column action.address unique values: 
[nan]
column action.balance unique values: 
[nan]
column action.callType unique values: 
['call']
column action.from unique values: 
['0xdc2d516c2eb8741f12df5b82e73a593c25e7f2da'
 '0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2'
 '0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f']
column action.gas unique values: 
['0x6d20dc' '0x6b164c' '0x694ac5' '0x8fc' '0x6aaa87' '0x68e0af' '0x6a3ec2'
 '0x687699' '0x69d2fc' '0x680c82' '0x696737' '0x67a26c' '0x68fb72'
 '0x673856' '0x688fad' '0x66ce40' '0x6823e8' '0x66642b' '0x67b823'
 '0x65fa15' '0x674c5e' '0x658fff' '0x66e099' '0x6525e9' '0x6674d4'
 '0x64bbd3' '0x66090f' '0x6451bd' '0x659d4a' '0x63e7a7' '0x653185'
 '0x637d91' '0x64c5c0' '0x63137b' '0x6459fa' '0x62a964' '0x63ee35'
 '0x623f4e' '0x638270' '0x61d538' '0x6316ab' '0x616b23' '0x62aae6'
 '0x61010d' '0x623f21' '0x6096f7' '0x61d35c' '0x602ce1' '0x616797'
 '0x5fc2cb' '0x60fbd2' '0x5f58b5' '0x60900d' '0x5eee9f' '0x602448'
 '0x5e8489' '0x5fb883' '0x5e1a73' '0x5f4cbe' 

What we could also see are a lot of transactions with a trace length of 451. Let's look into that as well to see, what is happening there

In [62]:
transaction_lookup[transaction_lookup['id'] == 2081526]

,transaction_hash,id
2081526,0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,2081526


In [63]:
b = raw_input[raw_input['transactionHash']=='0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8']

In [64]:
for column in b.columns:
    print("column "+column+" unique values: \n"+str(b[column].unique()))

column action.address unique values: 
[nan]
column action.balance unique values: 
[nan]
column action.callType unique values: 
['call']
column action.from unique values: 
['0xfb35694e93693be4cabfa7952979c3027eb22645'
 '0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2'
 '0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f']
column action.gas unique values: 
['0x612302' '0x5f4869' '0x5dac1a' '0x8fc' '0x5edca4' '0x5d4204' '0x5e70df'
 '0x5cd7ee' '0x5e051a' '0x5c6dd8' '0x5d9955' '0x5c03c2' '0x5d2d90'
 '0x5b99ac' '0x5cc1cb' '0x5b2f96' '0x5c5606' '0x5ac580' '0x5bea41'
 '0x5a5b6a' '0x5b7e7b' '0x59f153' '0x5b12b6' '0x59873d' '0x5aa6f1'
 '0x591d27' '0x5a3b2c' '0x58b312' '0x59cf67' '0x5848fc' '0x5963a2'
 '0x57dee6' '0x58f7dd' '0x5774d0' '0x588c18' '0x570aba' '0x582053'
 '0x56a0a4' '0x57b48e' '0x56368e' '0x5748c9' '0x55cc78' '0x56dd04'
 '0x556262' '0x56713e' '0x54f84b' '0x560579' '0x548e35' '0x5599b4'
 '0x54241f' '0x552def' '0x53ba09' '0x54c22a' '0x534ff4' '0x545665'
 '0x52e5de' '0x53eaa0' '0x527bc8' '0x537edb' 

In [65]:
b

,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,...,result,result.address,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type
757541,NaN,NaN,call,0xfb35694e93693be4cabfa7952979c3027eb22645,0x612302,NaN,0xe5927787000000000000000000000000e0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f0000000000000000000000...,NaN,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x0,...,NaN,NaN,NaN,0x40a264,0x,150,[],0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757542,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x5f4869,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[0],0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757543,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x5dac1a,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[0, 0]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757544,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[0, 1]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757545,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x5edca4,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[1],0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757546,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x5d4204,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[1, 0]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757547,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[1, 1]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757548,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x5e70df,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[2],0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757549,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x5cd7ee,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[2, 0]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call
757550,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[2, 1]",0xf4462ddd8f05f2683f58ce23b9b786385832a31842ae9ca9d3e1eb64f8989da8,41.0,call


In [66]:
a


,action.address,action.balance,action.callType,action.from,action.gas,action.init,action.input,action.refundAddress,action.to,action.value,...,result,result.address,result.code,result.gasUsed,result.output,subtraces,traceAddress,transactionHash,transactionPosition,type
859905,NaN,NaN,call,0xdc2d516c2eb8741f12df5b82e73a593c25e7f2da,0x6d20dc,NaN,0xe5927787000000000000000000000000e0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f0000000000000000000000...,NaN,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x0,...,NaN,NaN,NaN,0x56046a,0x,200,[],0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859906,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x6b164c,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[0],0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859907,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x694ac5,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[0, 0]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859908,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[0, 1]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859909,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x6aaa87,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[1],0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859910,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x68e0af,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[1, 0]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859911,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[1, 1]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859912,NaN,NaN,call,0xd86a66745ec6b9bf573fcb98a7ea4d39feb750d2,0x6a3ec2,NaN,0x828f1b42000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d2,NaN,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x0,...,NaN,NaN,NaN,0x671b,0x,2,[2],0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859913,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x687699,NaN,0x40c10f19000000000000000000000000d86a66745ec6b9bf573fcb98a7ea4d39feb750d20000000000000000000000...,NaN,0x6c170dc797014ea4e0900df58e4af67674ca54e8,0x0,...,NaN,NaN,NaN,0x3ad1,0x0000000000000000000000000000000000000000000000000000000000000001,0,"[2, 0]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
859914,NaN,NaN,call,0xe0d5ef9dbd7147e2d133aa8cb6d6c53778228e4f,0x8fc,NaN,0x,NaN,0x0000000000000000000000000000000000000000,0x0,...,NaN,NaN,NaN,0x0,0x,0,"[2, 1]",0xdeea0fd8d1eae3c9086537e71ca497ed98d8810e83d9a3aa4803c391a82512ba,22.0,call
